In [1]:
import json
from ultralytics import YOLO
import cv2
import numpy as np
import time

from matplotlib import pyplot as plt

In [2]:
class Back_number_recogniser:
    def __init__(self):
        pass
    def read_participants(self,classesFilePath):
        # Opening JSON file
        f = open(classesFilePath)
        
        
        self.number_to_name = {}
        # returns JSON object as 
        # a dictionary
        self.participants = json.load(f)
        for i in self.participants:
            self.number_to_name[i["race_rider_bib_number"]] = i["rider"]["first_name"] + " "+ i["rider"]["last_name"]
        # Closing file
        f.close()
        
    def load_detection_model(self, model_path = "C:/Users/Eigenaar/Documents/school_Quinten/unief_2023_2024/Masterthesis/implementatie/number_detection/number_detection_10epochs.pt"):
        device = torch.device("cuda")
        self.detection_model = YOLO(model_path)
        self.detection_model.to(device)
        
    def load_recognition_model(self, model_path = "C:/Users/Eigenaar/Documents/school_Quinten/unief_2023_2024/Masterthesis/implementatie/number_recognition/number_recognition_10epochs.pt"):
        device = torch.device("cuda")
        self.recognition_model = YOLO(model_path)
        self.recognition_model.to(device)
        
    def detect_numbers(self,image):
        
        results = self.detection_model(image,verbose=False)
        return results
    
    def add_bb(self, image, boxes):
        ret_image = np.copy(image)
        for i in boxes:
            top = tuple([int(i[0].item()),int(i[1].item())])
            bottom = tuple([int(i[2].item()),int(i[3].item())])
            cv2.rectangle(ret_image,top,bottom, (255,0,0), 2)
        return ret_image
    
    def crop_image(self, image, box):
        image_copy = np.copy(image)
        left = int(box[0].item())
        right = int(box[2].item())
        top = int(box[1].item())
        bottom = int(box[3].item())
        cropped_image = image_copy[ top:bottom, left:right] # Slicing to crop the image
        return cropped_image
    
    def deinterlace_image(self, image):
        ret_image = np.copy(image)
        for i in range(len(ret_image)//2):
            ret_image[2*i+1] = ret_image[2*i]
        return ret_image
    
    def recognise_numbers(self, image, bboxes):
        cropped_images = []
        for i in range(len(bboxes)):
            cropped_image = self.crop_image(image, bboxes[i])
            deinterlaced_image = self.deinterlace_image(cropped_image)
            cropped_images.append(deinterlaced_image)
        
        results = self.recognition_model(cropped_images, verbose=False)
        detected_numbers = []
        for i in range(len(results)):
            result = results[i].boxes
            numbers = []
            coords = []
            for j in range(len(result.cls)):
                numbers.append(result.cls[j].item())
                coords.append(result.xyxy[j][0].item())
            numbers_sorted = [x for _,x in sorted(zip(coords,numbers))]
            number = ''
            for j in numbers_sorted:
                number+= str(int(j))
            detected_numbers.append(number)
            print(f"numbers in cropped_image: {numbers} and with coordinates {coords}")
        return detected_numbers
        
        
    def annotate_image(self, image, boxes,numbers,threshold = 0.5):
        
        imH, imW, imC = image.shape
        annotated_image = self.add_bb(image, boxes)
        
        if len(boxes) != 0:
            for i in range(len(boxes)):
                
                classLabelText = numbers[i]
                left = int(boxes[i][0].item())
                top = int(boxes[i][1].item())
                cv2.putText(annotated_image, classLabelText, (left, top-10), cv2.FONT_HERSHEY_PLAIN, 1, [0,0,255],2)
                
        return annotated_image
                
        
    def predict_image(self,image):
        results = self.detect_numbers(image)
        boxes = results[0].boxes.xyxy
        if len(boxes) != 0:
            boxed_image = self.add_bb(image, boxes)
            detected_numbers = self.recognise_numbers(image,boxes)
            # print(detected_numbers)
            annotated_image = self.annotate_image(image, boxes, detected_numbers)
            return annotated_image
        else:
            return image
    
    def process_image(self,imagePath):
        image = cv2.imread(imagePath)
        annotated_image = self.predict_image(image)
        # cv2.imshow("result", annotated_image)
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()
        return annotated_image
    
    def process_video(self,videoPath,outputfile,startpoint = 0):
        cap = cv2.VideoCapture(videoPath)
        cap.set(cv2.CAP_PROP_POS_FRAMES, startpoint)
        # Check if the video capture was successful
        if not cap.isOpened():
            print("Error: Could not open the video file.")
            exit()
        frame_width = int(frame.shape[1]) 
        frame_height = int(frame.shape[0])
        size = (frame_width, frame_height)
        #result = cv2.VideoWriter(outputfile,  cv2.VideoWriter_fourcc(*'MJPG'), 25, size) 
        # used to record the time when we processed last frame 
        prev_frame_time = time.time() 
          
        # used to record the time at which we processed current frame 
        new_frame_time = 0
        while True:
            ret, frame = cap.read()
            print(frame)
            processed_frame = self.predict_image(frame)
            # font which we will be using to display FPS 
            font = cv2.FONT_HERSHEY_SIMPLEX 
            # time when we finish processing for this frame 
            new_frame_time = time.time() 
          
            # Calculating the fps 
          
            # fps will be number of frame processed in given time frame 
            # since their will be most of time error of 0.001 second 
            # we will be subtracting it to get more accurate result 
            fps = 1/(new_frame_time-prev_frame_time) 
            prev_frame_time = new_frame_time 
          
            # converting the fps into integer 
            fps = int(fps) 
          
            # converting the fps to string so that we can display it on frame 
            # by using putText function 
            fps = str(fps) 
          
            # putting the FPS count on the frame 
            cv2.putText(processed_frame, fps, (7, 70), font, 3, (100, 255, 0), 3, cv2.LINE_AA) 
            plt.imshow(processed_frame)
            plt.show()
            break
            #result.write(processed_frame)
            #cv2.imshow('frame', processed_frame)
            
            #if cv2.waitKey(1) == ord('q'):
             #   break
        cap.release()
        #result.release()
        #cv2.destroyAllWindows()

In [ ]:

#image = "C:/Users/Eigenaar/Documents/school_Quinten/unief_2023_2024/Masterthesis/data/wva_foto.png"
#image = "C:/Users/Eigenaar/Documents/school_Quinten/unief_2023_2024/Masterthesis/implementatie/number_detection/data/images/e1948ebd-759b-11ee-bced-fcb3bc5cd112.jpg"
#image = "C:/Users/Eigenaar/Documents/school_Quinten/unief_2023_2024/Masterthesis/implementatie/number_detection/data/images/d2293da5-759a-11ee-b39f-fcb3bc5cd112.jpg"
#image = "C:/Users/Eigenaar/Documents/school_Quinten/unief_2023_2024/Masterthesis/implementatie/number_detection/data/images/d7a17471-759a-11ee-b7c3-fcb3bc5cd112.jpg"
#video = "C:/Users/Eigenaar/Documents/school_Quinten/unief_2023_2024/Masterthesis/implementatie/mt23/data/cut_rvv_1.mp4"
video = "data/cut_rvv_1.mp4"
#participant_file = "C:/Users/Eigenaar/Documents/school_Quinten/unief_2023_2024/Masterthesis/data/RVV23participants.json"
detection_path = "number_detection/runs/detect/train6/weights/best.pt" 
recognition_path = "number_recognition/runs/detect/train4/weights/best.pt"
outputfile = "data/new_results_cut_rvv_1.mp4"

recogniser = Back_number_recogniser()
recogniser.read_participants(participant_file)
"""recogniser.load_detection_model(detection_path)
recogniser.load_recognition_model(recognizer_path)
recogniser.process_video(video,outputfile)"""


(1080, 1920, 3)


OpenCV: FFMPEG: tag 0x47504a4d/'MJPG' is not supported with codec id 7 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'

0: 384x640 6 back_numbers, 40.8ms
Speed: 4.9ms preprocess, 40.8ms inference, 2.7ms postprocess per image at shape (1, 3, 384, 640)

0: 640x640 1 1, 1 4, 1: 640x640 1 1, 1 4, 2: 640x640 1 7, 1 9, 3: 640x640 1 3, 1 7, 1 9, 4: 640x640 1 1, 1 6, 5: 640x640 1 6, 48.6ms
Speed: 2.4ms preprocess, 8.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)


numbers in cropped_image: [4.0, 1.0] and with coordinates [25.3617000579834, 8.026300430297852]
numbers in cropped_image: [4.0, 1.0] and with coordinates [28.080873489379883, 2.816702127456665]
numbers in cropped_image: [7.0, 9.0] and with coordinates [50.227603912353516, 10.582493782043457]
numbers in cropped_image: [9.0, 7.0, 3.0] and with coordinates [5.561092376708984, 50.66638946533203, 6.307968616485596]
numbers in cropped_image: [6.0, 1.0] and with coordinates [15.4024658203125, 40.06043243408203]
numbers in cropped_image: [6.0] and with coordinates [12.724855422973633]
['14', '14', '97', '937', '61', '6']



0: 384x640 6 back_numbers, 10.1ms
Speed: 2.6ms preprocess, 10.1ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 640x640 1 7, 1 9, 1: 640x640 1 1, 1 4, 2: 640x640 1 1, 1 4, 3: 640x640 1 1, 1 6, 4: 640x640 1 7, 1 9, 5: 640x640 1 6, 14.1ms
Speed: 2.7ms preprocess, 2.4ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)


numbers in cropped_image: [9.0, 7.0] and with coordinates [15.502945899963379, 55.499794006347656]
numbers in cropped_image: [4.0, 1.0] and with coordinates [25.700105667114258, 8.752190589904785]
numbers in cropped_image: [4.0, 1.0] and with coordinates [29.898378372192383, 3.7897815704345703]
numbers in cropped_image: [6.0, 1.0] and with coordinates [10.401365280151367, 35.010982513427734]
numbers in cropped_image: [9.0, 7.0] and with coordinates [13.937320709228516, 58.19736099243164]
numbers in cropped_image: [6.0] and with coordinates [13.447564125061035]
['97', '14', '14', '61', '97', '6']


...


0: 384x640 4 back_numbers, 8.0ms
Speed: 1.8ms preprocess, 8.0ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 640x640 1 7, 1 9, 1: 640x640 1 1, 1 4, 2: 640x640 1 1, 1 4, 3: 640x640 1 1, 1 7, 1 9, 10.1ms
Speed: 2.0ms preprocess, 2.5ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)


numbers in cropped_image: [9.0, 7.0] and with coordinates [10.462775230407715, 54.16371154785156]
numbers in cropped_image: [1.0, 4.0] and with coordinates [1.2396106719970703, 42.55447006225586]
numbers in cropped_image: [4.0, 1.0] and with coordinates [30.06089973449707, 4.45157527923584]
numbers in cropped_image: [9.0, 1.0, 7.0] and with coordinates [8.237736701965332, 60.616546630859375, 60.07716369628906]
['97', '14', '14', '971']



0: 384x640 4 back_numbers, 10.1ms
Speed: 2.7ms preprocess, 10.1ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 640x640 1 7, 1 9, 1: 640x640 1 4, 1 7, 2: 640x640 1 1, 1 4, 3: 640x640 1 1, 1 7, 1 9, 10.0ms
Speed: 2.1ms preprocess, 2.5ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)
WARNING ⚠️ 'source' is missing. Using 'source=/opt/conda/lib/python3.11/site-packages/ultralytics/assets'.


numbers in cropped_image: [9.0, 7.0] and with coordinates [10.299071311950684, 54.786773681640625]
numbers in cropped_image: [4.0, 7.0] and with coordinates [32.011722564697266, 4.465698719024658]
numbers in cropped_image: [4.0, 1.0] and with coordinates [46.414669036865234, 5.030961513519287]
numbers in cropped_image: [9.0, 1.0, 7.0] and with coordinates [9.669785499572754, 64.26592254638672, 61.9906005859375]
['97', '74', '14', '971']



image 1/2 /opt/conda/lib/python3.11/site-packages/ultralytics/assets/bus.jpg: 640x480 (no detections), 56.5ms
image 2/2 /opt/conda/lib/python3.11/site-packages/ultralytics/assets/zidane.jpg: 384x640 (no detections), 8.2ms
Speed: 2.2ms preprocess, 32.4ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)
WARNING ⚠️ 'source' is missing. Using 'source=/opt/conda/lib/python3.11/site-packages/ultralytics/assets'.

image 1/2 /opt/conda/lib/python3.11/site-packages/ultralytics/assets/bus.jpg: 640x480 (no detections), 9.7ms
image 2/2 /opt/conda/lib/python3.11/site-packages/ultralytics/assets/zidane.jpg: 384x640 (no detections), 8.3ms
Speed: 2.3ms preprocess, 9.0ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)
WARNING ⚠️ 'source' is missing. Using 'source=/opt/conda/lib/python3.11/site-packages/ultralytics/assets'.

image 1/2 /opt/conda/lib/python3.11/site-packages/ultralytics/assets/bus.jpg: 640x480 (no detections), 9.1ms
image 2/2 /opt/conda/lib/python3.11/si